In [2]:
%pip install pandas numpy scikit-learn pyttsx3

  Using cached numpy-2.4.2-cp313-cp313-win_amd64.whl.metadata (6.6 kB)
  Using cached scikit_learn-1.8.0-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached pywin32-311-cp313-cp313-win_amd64.whl.metadata (10 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------  9.7/9.7 MB 61.5 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 54.2 MB/s  0:00:00
Using cached numpy-2.4.2-cp313-cp313-win_amd64.whl (12.3 MB)
Using cached scikit_learn-1.8.0-cp313-cp313-win_amd64.whl (8.0 MB)
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
   ---------------------------------------- 0.0/36.5 MB ? eta -:--:--
   ------------------------ --------------- 22.5/36.5 MB 113.1 MB/s eta 0:00:01
   -----------------------------


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Clean + Standardize columns (creates date, amount, merchant, label)

In [14]:
import numpy as np
import pandas as pd

# Load (use raw string or double slashes)
df = pd.read_csv(r"E:\GenerativeAiAssignment\Open Bank Transaction Data - Read-Only Original.csv")

# 1) Standardize date
df["date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")

# 2) Make sure debit/credit are numeric (many files store them as text with commas)
df["Debit Amount"] = pd.to_numeric(df["Debit Amount"], errors="coerce")
df["Credit Amount"] = pd.to_numeric(df["Credit Amount"], errors="coerce")

# 3) Create a single 'amount' column:
#    - If Credit exists → Income amount
#    - Else if Debit exists → Spend amount
df["amount"] = df["Credit Amount"].fillna(0) + df["Debit Amount"].fillna(0)

# 4) Create 'merchant' column
df["merchant"] = df["Transaction Description"].astype(str)

# 5) Create 'label' (Income vs Discretionary)
df["label"] = np.where(df["Credit Amount"].fillna(0) > 0, "Income", "Discretionary")

# 6) Keep only required columns + drop bad rows
df = df.dropna(subset=["date", "amount"])
df = df[df["amount"] != 0].copy()

df[["date","merchant","amount","label"]].head(), df["label"].value_counts()

C:\Users\shiru\AppData\Local\Temp\ipykernel_19716\4254267613.py:8: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")


(        date            merchant  amount          label
 0 2022-07-25     SAVE THE CHANGE    3.11  Discretionary
 1 2022-07-25  LIDL GB  NOTTINGHA   15.02  Discretionary
 2 2022-07-25   NON-GBP PURCH FEE    0.50  Discretionary
 3 2022-07-25   NON-GBP TRANS FEE    0.37  Discretionary
 4 2022-07-25       TRAVELIUM LLC   12.59  Discretionary,
 label
 Discretionary    6122
 Income            445
 Name: count, dtype: int64)

Monthly aggregation function (your prototype expects this)

In [15]:
def build_monthly_from_transactions(transactions: pd.DataFrame) -> pd.DataFrame:
    tx = transactions.copy()
    tx["date"] = pd.to_datetime(tx["date"])
    tx["year_month"] = tx["date"].dt.to_period("M").astype(str)

    pivot = (
        tx.pivot_table(index="year_month", columns="label", values="amount", aggfunc="sum")
        .fillna(0)
        .rename(columns={"Income": "income", "Fixed": "fixed_costs", "Discretionary": "discretionary"})
    )

    # Ensure expected columns exist
    for col in ["income", "fixed_costs", "discretionary"]:
        if col not in pivot.columns:
            pivot[col] = 0.0

    pivot = pivot.reset_index()
    pivot["month"] = pd.to_datetime(pivot["year_month"] + "-01").dt.month
    pivot["year"] = pd.to_datetime(pivot["year_month"] + "-01").dt.year

    # Lag/rolling (trend)
    pivot = pivot.sort_values("year_month").reset_index(drop=True)
    pivot["disc_lag1"] = pivot["discretionary"].shift(1)
    pivot["disc_roll3"] = pivot["discretionary"].rolling(3).mean()

    return pivot.dropna().reset_index(drop=True)

monthly_real = build_monthly_from_transactions(df)
monthly_real.head()

label,year_month,discretionary,income,fixed_costs,month,year,disc_lag1,disc_roll3
0,2015-09,3145.10,3539.89,0.0,9,2015,2803.04,2478.003333
1,2015-10,4328.20,3073.57,0.0,10,2015,3145.10,3425.446667
2,2015-11,2920.55,3244.39,0.0,11,2015,4328.20,3464.616667
3,2015-12,3365.97,4744.65,0.0,12,2015,2920.55,3538.240000
4,2016-01,3239.97,3132.44,0.0,1,2016,3365.97,3175.496667


Add a simple “Fixed vs Discretionary” rule

In [16]:
# Simple keyword rules for FIXED costs (edit keywords as needed)
fixed_keywords = [
    "rent", "mortgage", "insurance", "hydro", "electric", "water",
    "internet", "phone", "utility", "netflix", "spotify"
]

desc_lower = df["merchant"].str.lower()

is_fixed = desc_lower.apply(lambda x: any(k in x for k in fixed_keywords))

# Only apply to spending rows (not income)
df.loc[(df["label"] == "Discretionary") & (is_fixed), "label"] = "Fixed"

df["label"].value_counts()

label
Discretionary    6036
Income            445
Fixed              86
Name: count, dtype: int64

Ridge baseline model (time split)

In [17]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

FEATURES = ["income", "fixed_costs", "month", "disc_lag1", "disc_roll3"]

def train_ridge_time_split(monthly_df, alpha=1.0, train_frac=0.8):
    monthly_df = monthly_df.sort_values("year_month").reset_index(drop=True)
    split = int(len(monthly_df) * train_frac)

    X = monthly_df[FEATURES]
    y = monthly_df["discretionary"]

    X_train, X_test = X.iloc[:split], X.iloc[split:]
    y_train, y_test = y.iloc[:split], y.iloc[split:]

    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds) if len(y_test) else np.nan
    r2 = r2_score(y_test, preds) if len(y_test) > 1 else np.nan

    return model, mae, r2

model, mae, r2 = train_ridge_time_split(monthly_real)
print("MAE:", mae)
print("R² :", r2)

MAE: 414.56913974050224
R² : 0.26584065973170956


Alerts (percentile threshold)

In [18]:
def score_months(monthly_df, model, threshold_pct=0.82):
    scored = monthly_df.sort_values("year_month").reset_index(drop=True).copy()
    scored["baseline"] = model.predict(scored[FEATURES])
    scored["residual"] = scored["discretionary"] - scored["baseline"]

    threshold_value = scored["residual"].quantile(threshold_pct)
    scored["flag_review"] = scored["residual"] > threshold_value

    return scored, threshold_value

scored, threshold_value = score_months(monthly_real, model, threshold_pct=0.82)
print("Threshold:", threshold_value)
print("Alert rate:", round(scored["flag_review"].mean() * 100, 2), "%")

scored[["year_month","income","fixed_costs","discretionary","baseline","residual","flag_review"]].tail(12)

Threshold: 990.8347602502408
Alert rate: 18.07 %


label,year_month,income,fixed_costs,discretionary,baseline,residual,flag_review
71,2021-08,5735.23,0.0,5987.18,5761.936986,225.243014,False
72,2021-09,6815.70,0.0,6515.87,6722.216739,-206.346739,False
73,2021-10,5905.23,0.0,6149.22,5760.400962,388.819038,False
74,2021-11,5569.31,0.0,5606.01,5341.108197,264.901803,False
75,2021-12,5997.10,0.0,5586.10,5649.011340,-62.911340,False
76,2022-01,5302.43,0.0,5686.67,5863.847209,-177.177209,False
77,2022-02,4691.96,0.0,4627.71,5149.666614,-521.956614,False
78,2022-03,4895.86,0.0,4943.52,5255.916869,-312.396869,False
79,2022-04,4582.92,0.0,4636.03,4826.606546,-190.576546,False
80,2022-05,5188.15,0.0,4981.55,5350.649080,-369.099080,False


Deterministic driver extraction (merchant totals)

In [19]:
def top_drivers_for_month(transactions, year_month, top_k=3):
    tx = transactions.copy()
    tx["year_month"] = pd.to_datetime(tx["date"]).dt.to_period("M").astype(str)

    month_tx = tx[(tx["year_month"] == year_month) & (tx["label"] == "Discretionary")]
    if month_tx.empty:
        return []

    agg = month_tx.groupby("merchant", as_index=False)["amount"].sum().sort_values("amount", ascending=False)
    return list(zip(agg["merchant"].head(top_k).tolist(), agg["amount"].head(top_k).tolist()))

In [20]:
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class CoachingInputs:
    year_month: str
    baseline: float
    actual: float
    residual: float
    top_drivers: List[Tuple[str, float]]

def build_prompt(ci: CoachingInputs) -> str:
    drivers_text = ", ".join([f"{name} (£{amt:,.0f})" for name, amt in ci.top_drivers]) or "no dominant merchant"
    return f"""You are a privacy-preserving financial assistant for a visually impaired user.
Rules:
- Use ONLY the numbers provided. Do not invent amounts.
- Under 100 words.
- Calm, supportive tone.
Task: Explain what changed and why, then give ONE action.

Month: {ci.year_month}
Baseline discretionary: £{ci.baseline:,.0f}
Actual discretionary: £{ci.actual:,.0f}
Above baseline: £{ci.residual:,.0f}
Top drivers: {drivers_text}
"""

# Keep as stub OR replace with your GenAI tool call
def genai_generate_explanation(prompt: str) -> str:
    return (
        "This month your discretionary spending was higher than your usual baseline. "
        "The biggest changes came from a few specific transactions. "
        "If these were planned, you can mark this as expected. "
        "If not, consider reviewing those merchants and setting a soft limit next month."
    )

# Generate scripts for first few flagged months
flagged_months = scored.loc[scored["flag_review"], "year_month"].tolist()

scripts = []
for ym in flagged_months[:5]:
    row = scored.loc[scored["year_month"] == ym].iloc[0]
    drivers = top_drivers_for_month(df, ym, top_k=3)

    ci = CoachingInputs(
        year_month=ym,
        baseline=float(row["baseline"]),
        actual=float(row["discretionary"]),
        residual=float(row["residual"]),
        top_drivers=drivers
    )

    prompt = build_prompt(ci)
    script = genai_generate_explanation(prompt)
    scripts.append({"year_month": ym, "script": script})

pd.DataFrame(scripts)

,year_month,script
0,2015-10,This month your discretionary spending was hig...
1,2016-07,This month your discretionary spending was hig...
2,2016-11,This month your discretionary spending was hig...
3,2017-06,This month your discretionary spending was hig...
4,2017-09,This month your discretionary spending was hig...


In [21]:
scored.to_csv("seemycash_scored_months.csv", index=False)
pd.DataFrame(scripts).to_csv("seemycash_genai_scripts.csv", index=False)

print("Saved: seemycash_scored_months.csv and seemycash_genai_scripts.csv")

Saved: seemycash_scored_months.csv and seemycash_genai_scripts.csv
